In [1]:
#upgrade seaborn
#import sys
#!{sys.executable} -m pip install -U seaborn


In [10]:
# Import libraries
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier

# classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

# evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from scipy.stats import chisquare, chi2_contingency

In [11]:
# load datasest
df = pd.read_excel("./data/sarscov_2.xlsx", header=1, comment="#")

In [12]:
# cleanup
indexNames = df[df['Age'] == 'NS'].index
 
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

# set age to numeric
df["Age"] = df["Age"].astype(int)

# bin ages
df["Age bin"] = pd.cut(x=df['Age'], 
                       bins=[10, 20, 29, 39, 49, 59, 69, 79, 89], 
                       labels=['10s', '20s', '30s', '40s', '50s','60s', '70s', '80s'])

# remove unwanted spaces

df[" IgM/Pos."] = df[" IgM/Pos."].str.replace(' ', '')

In [13]:
# create class column
def get_class(row):
    wanted = row["lab-ID"]
    return wanted.split("-")[0]

df["status"] = df.apply(get_class, axis=1)

# create target column
def encode_symptoms(row):
    wanted = row["Symptomes"]
    if str(wanted) == "no":
        return 0
    return 1

df["target"] = df.apply(encode_symptoms, axis=1)

In [36]:
df

,lab-ID,Gender,Age,IgM/Pos.,IgM/b inten.,IgM/b inten. value,IgG/Pos.,IgG/b inten.,IgG/b inten. value,Symptomes,Age bin,status,target
0,LSS-1,M,54,P,F,10,P,S,100,"Headache, sorethroat",50s,LSS,1
1,LSS-2,F,45,N,no,0,N,no,0,no,40s,LSS,0
2,LSS-3,F,37,N,no,0,N,no,0,"Fatigue, headache, abdominal pain, diarrhoea",30s,LSS,1
3,LSS-4,M,40,N,no,0,N,no,0,no,40s,LSS,0
4,LSS-5,M,50,N,no,0,P,F,10,no,50s,LSS,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,HSS-102,F,40,N,no,0,N,no,0,"Sorthroat, headache",40s,HSS,1
102,HSS-103,F,37,P,VF,5,P,F,10,Sorthroat,30s,HSS,1
103,HSS-104,F,60,N,no,0,N,no,0,no,60s,HSS,0
104,HSS-105,M,50,P,S,100,P,S,100,"Fatigue, abdominal pain, anosmia, headache, di...",50s,HSS,1


In [14]:
targets = df["target"].values

In [15]:
# encodings

# bulk encodings

wanted_columns = ["Gender", " IgM/Pos.",
                   " IgM/b inten.", "IgG/Pos.",
                   " IgG/b inten.", "status", "Age bin"]
df_wanted = df[wanted_columns]

le = LabelEncoder()
df_wanted_le = df_wanted.apply(le.fit_transform)

ohe = OneHotEncoder()
ohe.fit(df_wanted_le)
input_df = ohe.transform(df_wanted_le).toarray()
input_df.shape
    

(105, 23)

In [16]:
input_df

array([[0., 1., 0., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

In [17]:
# Generate training and test sets

X_train, X_test, y_train, y_test = train_test_split(input_df,
                                                    targets,
                                                    random_state=42)



In [22]:
# Benchmark with dummy classifier

dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_test, y_test)



0.5925925925925926

In [20]:
# Random forest model

model = RandomForestClassifier(n_jobs=2, random_state=42)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.5925925925925926


In [23]:
model.feature_importances_

array([0.07294861, 0.06746047, 0.02559584, 0.02739642, 0.01928097,
       0.00056769, 0.00825625, 0.02396342, 0.08069704, 0.07951529,
       0.02687549, 0.03814273, 0.00388376, 0.10130147, 0.07067969,
       0.06613774, 0.02375316, 0.04235328, 0.04428636, 0.06233487,
       0.07256331, 0.02588711, 0.01611904])

In [104]:
# Set up different classifiers

names = ["K-Nearest Neighbors", "Naive Bayes", "Linear SVM", "RBF SVM", 
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost"]

classifiers = [
    KNeighborsClassifier(),
    GaussianNB(),
    SVC(kernel="linear", C=0.025),
    SVC(gamma="scale", C=1),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(),
    AdaBoostClassifier()]



In [105]:
# Compare different classifiers

for name, clf in zip(names, classifiers):
    model = clf
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    report = classification_report(y_test, y_pred)
    print("###### " + name + " #######")
    print("accuracy: {}".format(accuracy))
    print("report:")
    print(report)
    print("------------------------------\n")


###### K-Nearest Neighbors #######
accuracy: 0.5925925925925926
report:
              precision    recall  f1-score   support

           0       0.33      0.57      0.42         7
           1       0.80      0.60      0.69        20

    accuracy                           0.59        27
   macro avg       0.57      0.59      0.55        27
weighted avg       0.68      0.59      0.62        27

------------------------------

###### Naive Bayes #######
accuracy: 0.48148148148148145
report:
              precision    recall  f1-score   support

           0       0.23      0.43      0.30         7
           1       0.71      0.50      0.59        20

    accuracy                           0.48        27
   macro avg       0.47      0.46      0.44        27
weighted avg       0.59      0.48      0.51        27

------------------------------

###### Linear SVM #######
accuracy: 0.5185185185185185
report:
              precision    recall  f1-score   support

           0       0.25    

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [24]:
# chi squared test

# Null Hypothesis -> the relative proportion of  positive IgM positive samples 
#                    is the same as IgG irrespective of social class

igm_df = df[df[" IgM/Pos."] == "P"].groupby(['status', ' IgM/Pos.']).size().reset_index(name='IgM pos counts').drop([" IgM/Pos."],
                                                                                                                   axis=1)

igg_df = df[df["IgG/Pos."] == "P"].groupby(['status', 'IgG/Pos.']).size().reset_index(name='IgG pos counts')

igg_count = igg_df["IgG pos counts"]

pivot_df = igm_df.join(igg_count)


In [25]:
pivot_df

,status,IgM pos counts,IgG pos counts
0,HSS,12,26
1,LSS,12,33


In [27]:
pivot_df = pivot_df.set_index("status")

In [30]:
result = chi2_contingency(pivot_df)

In [35]:
statistic = result[0]
statistic

0.06191510143060089

In [34]:
p_value = result[1]
p_value

0.8034942543245844